In [20]:
from SBM_SDE import *
from obs_and_flow_classes_and_functions import *
import seaborn as sns
import torch
from torch import nn
import torch.distributions as d
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import math
from tqdm import tqdm
import random
from torch.autograd import Function
import argparse
import os
import sys
from pathlib import Path
import shutil
import pandas as pd

In [21]:
torch.manual_seed(0)
devi = torch.device("".join(["cuda:",f'{cuda_id}']) if torch.cuda.is_available() else "cpu")

cuda_id = 1
dt = .2 #SDE discretization timestep.
t = 500 #Simulation run for T hours.
n = int(t / dt) 
t_span = np.linspace(0, t, n + 1)
t_span_tensor = torch.reshape(torch.Tensor(t_span), [1, n + 1, 1]) #T_span needs to be converted to tensor object. Additionally, facilitates conversion of I_S and I_D to tensor objects.
l_r = 1e-3
niter = 11
piter = 5
batch_size = 2 #Number of sets of observation outputs to sample per set of parameters.
state_dim_SCON = 3 #Not including CO2 in STATE_DIM, because CO2 is an observation.
state_dim_SAWB = 4 #Not including CO2 in STATE_DIM, because CO2 is an observation.

In [36]:
temp_ref = 283

#System parameters from deterministic CON model
u_M = 0.002
a_SD = 0.33
a_DS = 0.33
a_M = 0.33
a_MSC = 0.5
k_S_ref = 0.000025
k_D_ref = 0.005
k_M_ref = 0.0002
Ea_S = 75
Ea_D = 50
Ea_M = 50

#SCON diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
s_SOC = 0.1
s_DOC = 0.1
s_MBC = 0.1

SCON_C_params_dict = {'u_M': u_M, 'a_SD': a_SD, 'a_DS': a_DS, 'a_M': a_M, 'a_MSC': a_MSC, 'k_S_ref': k_S_ref, 'k_D_ref': k_D_ref, 'k_M_ref': k_M_ref, 'Ea_S': Ea_S, 'Ea_D': Ea_D, 'Ea_M': Ea_M, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC}
SCON_SS_params_dict = {'u_M': u_M, 'a_SD': a_SD, 'a_DS': a_DS, 'a_M': a_M, 'a_MSC': a_MSC, 'k_S_ref': k_S_ref, 'k_D_ref': k_D_ref, 'k_M_ref': k_M_ref, 'Ea_S': Ea_S, 'Ea_D': Ea_D, 'Ea_M': Ea_M, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC}

#System parameters from deterministic AWB model
u_Q_ref = 0.2
Q = 0.002
a_MSA = 0.5
K_D = 200
K_U = 1
V_D_ref = 0.4
V_U_ref = 0.02
Ea_V_D = 75
Ea_V_U = 50
r_M = 0.0004
r_E = 0.00001
r_L = 0.0005

#SAWB diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
c_EEC = 0.001
s_SOC = 0.1
s_DOC = 0.1
s_MBC = 0.1
s_EEC = 0.1

SAWB_C_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_D': K_D, 'K_U': K_U, 'V_D_ref': V_D_ref, 'V_U_ref': V_U_ref, 'Ea_V_D': Ea_V_D, 'Ea_V_U': Ea_V_U, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC, 'c_EEC': c_EEC}
SAWB_SS_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_D': K_D, 'K_U': K_U, 'V_D_ref': V_D_ref, 'V_U_ref': V_U_ref, 'Ea_V_D': Ea_V_D, 'Ea_V_U': Ea_V_U, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC, 's_EEC': s_EEC}

#System parameters from deterministic model
u_Q_ref = 0.2
Q = 0.002
a_MSA = 0.5
K_DE = 200
K_UE = 1
V_DE_ref = 0.4
V_UE_ref = 0.02
Ea_V_DE = 75
Ea_V_UE = 50
r_M = 0.0004
r_E = 0.00001
r_L = 0.0005

#Diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
c_EEC = 0.001
s_SOC = 0.1
s_DOC = 0.1
s_MBC = 0.1
s_EEC = 0.1

SAWB_ECA_C_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_DE': K_DE, 'K_UE': K_UE, 'V_DE_ref': V_DE_ref, 'V_UE_ref': V_UE_ref, 'Ea_V_DE': Ea_V_DE, 'Ea_V_UE': Ea_V_UE, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC, 'c_EEC': c_EEC}
SAWB_ECA_SS_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_DE': K_DE, 'K_UE': K_UE, 'V_DE_ref': V_DE_ref, 'V_UE_ref': V_UE_ref, 'Ea_V_DE': Ea_V_DE, 'Ea_V_UE': Ea_V_UE, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC, 's_EEC': s_EEC}

In [37]:
#System parameters from deterministic AWB model
u_Q_ref = 0.2
Q = 0.002
a_MSA = 0.5
K_D = 200
K_U = 1
V_D_ref = 0.4
V_U_ref = 0.02
Ea_V_D = 75
Ea_V_U = 50
r_M = 0.0004
r_E = 0.00001
r_L = 0.0005

#SAWB diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
c_EEC = 0.001

SAWB_C_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_D': K_D, 'K_U': K_U, 'V_D_ref': V_D_ref, 'V_U_ref': V_U_ref, 'Ea_V_D': Ea_V_D, 'Ea_V_U': Ea_V_U, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC, 'c_EEC': c_EEC}

In [38]:
#Obtain SOC and DOC pool litter inputs for all SBMs.
i_s_tensor = 0.001 + 0.0005 * torch.sin((2 * np.pi / (24 * 365)) * t_span_tensor) #Exogenous SOC input function
i_d_tensor = 0.0001 + 0.00005 * torch.sin((2 * np.pi / (24 * 365)) * t_span_tensor) #Exogenous DOC input function

In [39]:
def calc_log_lik(C_PATH, T_SPAN_TENSOR, DT, I_S_TENSOR, I_D_TENSOR, DRIFT_DIFFUSION, PARAMS_DICT, TEMP_GEN, TEMP_REF):
    drift, diffusion_sqrt = DRIFT_DIFFUSION(C_PATH[:, :-1, :], T_SPAN_TENSOR[:, :-1, :], I_S_TENSOR[:, :-1, :], I_D_TENSOR[:, :-1, :], PARAMS_DICT, TEMP_GEN, TEMP_REF)
    euler_maruyama_state_sample_object = d.multivariate_normal.MultivariateNormal(loc = C_PATH[:, :-1, :] + drift * DT, scale_tril = diffusion_sqrt * math.sqrt(DT))
    return euler_maruyama_state_sample_object.log_prob(C_PATH[:, 1:, :]).sum(-1)

In [40]:
def train(DEVICE, L_R, NITER, PRETRAIN_ITER, BATCH_SIZE, SDEFLOW, ObsModel, csv_to_obs_df, DATA_CSV, OBS_ERROR_SCALE, STATE_DIM, T, DT, N, T_SPAN_TENSOR, I_S_TENSOR, I_D_TENSOR, DRIFT_DIFFUSION, PARAMS_DICT, TEMP_GEN, TEMP_REF, ANALYTICAL_STEADY_STATE_INIT):
    obs_times, obs_means, obs_error = csv_to_obs_df(DATA_CSV, STATE_DIM + 1, T, OBS_ERROR_SCALE) 
    obs_model = ObsModel(DEVICE, obs_times, DT, obs_means[:-1, :], obs_error[:, :-1]) #Hack for bypassing ObsModel and SDEFlow dimension mismatch issue.
    net = SDEFlow(DEVICE, BATCH_SIZE, obs_model, STATE_DIM, T, DT, N).to(DEVICE)
    optimizer = optim.Adam(net.parameters(), lr = L_R) 
    if PRETRAIN_ITER >= NITER:
        raise Exception("PRETRAIN_ITER must be < NITER.")
    best_loss_norm = 1e10
    best_loss_ELBO = 1e20
    norm_losses = [best_loss_norm] * 10
    ELBO_losses = [best_loss_ELBO] * 10
    C0 = ANALYTICAL_STEADY_STATE_INIT(I_S_TENSOR[0, 0, 0].item(), I_D_TENSOR[0, 0, 0].item(), PARAMS_DICT) #Calculate deterministic initial conditions.
    C0 = C0[(None,) * 2].repeat(BATCH_SIZE, 1, 1).to(DEVICE) #Assign initial conditions to C_PATH.
    with tqdm(total = NITER, desc = f'Train Diffusion', position = -1) as tq:
        for iter in range(NITER):
            net.train()
            optimizer.zero_grad()
            C_PATH, log_prob = net() #Obtain paths with solutions at times after t0.
            C_PATH = torch.cat([C0, C_PATH], 1) #Append deterministic CON initial conditions conditional on parameter values to C path. 
            if iter <= PRETRAIN_ITER:
                l1_norm_element = C_PATH - torch.mean(obs_model.mu, -1)
                l1_norm = torch.sum(torch.abs(l1_norm_element)).mean()
                best_loss_norm = l1_norm if l1_norm < best_loss_norm else best_loss_norm
                norm_losses.append(l1_norm.item())
                #l2_norm_element = C_PATH - torch.mean(obs_model.mu, -1)
                #l2_norm = torch.sqrt(torch.sum(torch.square(l2_norm_element))).mean()
                #best_loss_norm = l2_norm if l2_norm < best_loss_norm else best_loss_norm
                #l2_norm.backward()
                #norm_losses.append(l2_norm.item())
                if len(norm_losses) > 10:
                    norm_losses.pop(0)
                if iter % 10 == 0:
                    print(f"Moving average norm loss at {iter} iterations is: {sum(norm_losses) / len(norm_losses)}. Best norm loss value is: {best_loss_norm}.")
                    print('\nC_PATH mean =', C_PATH.mean(-2))
                    print('\nC_PATH =', C_PATH)
                l1_norm.backward()
            else:
                log_lik = calc_log_lik(C_PATH, T_SPAN_TENSOR.to(DEVICE), dt, I_S_TENSOR.to(DEVICE), I_D_TENSOR.to(DEVICE), DRIFT_DIFFUSION, PARAMS_DICT, TEMP_GEN, TEMP_REF)
                neg_ELBO = -log_lik.mean() - obs_model(C_PATH) + log_prob.mean()
                best_loss_ELBO = neg_ELBO if neg_ELBO < best_loss_ELBO else best_loss_ELBO
                ELBO_losses.append(neg_ELBO.item())
                if len(ELBO_losses) > 10:
                    ELBO_losses.pop(0)
                if iter % 10 == 0:
                    print(f"Moving average ELBO loss at {iter} iterations is: {sum(ELBO_losses) / len(ELBO_losses)}. Best ELBO loss value is: {best_loss_ELBO}.")
                    print('\nC_PATH mean =', C_PATH.mean(-2))
                    print('\n C_PATH =', C_PATH)
                neg_ELBO.backward()
            torch.nn.utils.clip_grad_norm_(net.parameters(), 3.0)
            optimizer.step()
            if iter % 100000 == 0 and iter > 0:
                optimizer.param_groups[0]['lr'] *= 0.1
            tq.update()

In [34]:
train(devi, l_r, niter, piter, batch_size, SDEFlow, ObsModel, csv_to_obs_df, 'CON_synthetic_sol_df.csv', 0.1, state_dim_SCON, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SCON_C, SCON_C_params_dict, temp_gen, temp_ref, analytical_steady_state_init_CON)


Train Diffusion:   0%|          | 0/11 [00:00<?, ?it/s]

Moving average norm loss at 0 iterations is: 9000022787.378124. Best norm loss value is: 227873.78125.

C_PATH mean = tensor([[0.8623, 0.6865, 0.9656],
        [0.8605, 0.6887, 0.9505]], grad_fn=<MeanBackward1>)

C_PATH = tensor([[[45.6603,  0.0715,  0.7147],
         [ 1.1695,  1.0302,  1.3381],
         [ 1.3118,  0.6691,  0.8699],
         ...,
         [ 1.4350,  0.7311,  1.6996],
         [ 0.9785,  0.6912,  1.5259],
         [ 0.6916,  1.2553,  0.6581]],

        [[45.6603,  0.0715,  0.7147],
         [ 0.8073,  0.1717,  0.7183],
         [ 1.0228,  0.3018,  0.4433],
         ...,
         [ 0.7979,  0.7958,  0.5515],
         [ 0.7442,  0.6323,  0.4608],
         [ 0.4203,  0.7186,  0.4649]]], grad_fn=<CatBackward>)



Train Diffusion:  91%|█████████ | 10/11 [00:26<00:02,  2.69s/it]

Moving average ELBO loss at 10 iterations is: 5.000000000000011e+19. Best ELBO loss value is: 163844.40625.

C_PATH mean = tensor([[2.4989, 0.5310, 0.6211],
        [2.5436, 0.5317, 0.6293]], grad_fn=<MeanBackward1>)

 C_PATH = tensor([[[45.6603,  0.0715,  0.7147],
         [ 1.5063,  0.5293,  0.3424],
         [ 0.8195,  2.0151,  0.3774],
         ...,
         [ 0.4991,  0.3210,  1.1009],
         [ 3.1494,  0.7787,  0.6943],
         [ 3.8478,  0.5191,  0.6199]],

        [[45.6603,  0.0715,  0.7147],
         [ 0.6080,  3.5804,  0.5559],
         [ 2.3427,  0.3734,  0.8063],
         ...,
         [ 4.1888,  0.8438,  0.2477],
         [ 1.1030,  0.4200,  0.3198],
         [ 1.0795,  0.4818,  0.2937]]], grad_fn=<CatBackward>)



Train Diffusion: 100%|██████████| 11/11 [00:28<00:00,  2.63s/it]


In [41]:
train(devi, l_r, niter, piter, batch_size, SDEFlow, ObsModel, csv_to_obs_df, 'CON_synthetic_sol_df.csv', 0.1, state_dim_SCON, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SCON_SS, SCON_SS_params_dict, temp_gen, temp_ref, analytical_steady_state_init_CON)


Train Diffusion:   0%|          | 0/11 [00:00<?, ?it/s]

Moving average norm loss at 0 iterations is: 9000022821.1. Best norm loss value is: 228211.0.

C_PATH mean = tensor([[0.6316, 0.6414, 0.6843],
        [0.6281, 0.6425, 0.6895]], grad_fn=<MeanBackward1>)

C_PATH = tensor([[[45.6603,  0.0715,  0.7147],
         [ 0.7824,  0.5765,  0.6740],
         [ 0.6466,  0.7676,  1.0582],
         ...,
         [ 0.7064,  0.7815,  0.6413],
         [ 0.8086,  0.6473,  0.6805],
         [ 0.4942,  0.6519,  0.4727]],

        [[45.6603,  0.0715,  0.7147],
         [ 0.3824,  0.5667,  0.6257],
         [ 0.5092,  0.4462,  0.5990],
         ...,
         [ 0.5274,  0.7122,  0.6839],
         [ 0.5412,  0.6507,  0.9572],
         [ 0.6354,  0.7999,  0.4718]]], grad_fn=<CatBackward>)



Train Diffusion:  91%|█████████ | 10/11 [00:27<00:02,  2.69s/it]

Moving average ELBO loss at 10 iterations is: 5.000000000000015e+19. Best ELBO loss value is: 118140.0390625.

C_PATH mean = tensor([[2.8900, 0.2827, 0.3689],
        [2.8040, 0.2808, 0.3569]], grad_fn=<MeanBackward1>)

 C_PATH = tensor([[[45.6603,  0.0715,  0.7147],
         [ 0.4044,  0.4504,  0.1998],
         [ 3.3811,  0.5256,  0.2236],
         ...,
         [ 0.9629,  0.3355,  0.8536],
         [ 1.0080,  0.2143,  0.2413],
         [ 0.7486,  0.3369,  0.2315]],

        [[45.6603,  0.0715,  0.7147],
         [ 1.3354,  0.6281,  0.5014],
         [ 0.9321,  0.3022,  0.3158],
         ...,
         [ 1.4652,  0.2599,  0.1690],
         [ 1.2888,  0.3166,  0.7326],
         [ 0.4505,  0.2339,  0.1618]]], grad_fn=<CatBackward>)



Train Diffusion: 100%|██████████| 11/11 [00:30<00:00,  2.74s/it]


In [42]:
train(devi, l_r, niter, piter, batch_size, SDEFlow, ObsModel, csv_to_obs_df, 'AWB_synthetic_sol_df.csv', 0.1, state_dim_SAWB, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SAWB_C, SAWB_C_params_dict, temp_gen, temp_ref, analytical_steady_state_init_AWB)


Train Diffusion:   0%|          | 0/11 [00:00<?, ?it/s]

Moving average norm loss at 0 iterations is: 9000027008.0. Best norm loss value is: 270080.0.

C_PATH mean = tensor([[0.8622, 0.9101, 0.6809, 0.5296],
        [0.8626, 0.9103, 0.6820, 0.5289]], grad_fn=<MeanBackward1>)

C_PATH = tensor([[[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02],
         [7.1490e-01, 6.2348e-01, 1.5707e+00, 5.6568e-01],
         [7.7201e-01, 7.5046e-01, 6.9551e-01, 5.8519e-01],
         ...,
         [7.6567e-01, 5.6933e-01, 5.4033e-01, 4.0968e-01],
         [7.4231e-01, 1.3486e+00, 6.5284e-01, 6.3531e-01],
         [5.7352e-01, 7.3884e-01, 5.9433e-01, 6.5786e-01]],

        [[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02],
         [1.2693e+00, 6.3619e-01, 5.3986e-01, 5.3479e-01],
         [1.5469e+00, 1.4846e+00, 1.1556e+00, 6.3247e-01],
         ...,
         [7.3338e-01, 1.1381e+00, 6.5717e-01, 4.3913e-01],
         [9.4472e-01, 5.9087e-01, 5.5797e-01, 5.7885e-01],
         [6.5255e-01, 1.3442e+00, 5.6849e-01, 5.7176e-01]]],
       grad_fn=<CatBackward>)



Train Diffusion:  91%|█████████ | 10/11 [00:28<00:02,  2.82s/it]

Moving average ELBO loss at 10 iterations is: 5.00000000000004e+19. Best ELBO loss value is: 696535.5.

C_PATH mean = tensor([[3.1215, 0.7630, 0.4007, 0.1884],
        [3.1075, 0.7578, 0.4014, 0.1897]], grad_fn=<MeanBackward1>)

 C_PATH = tensor([[[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02],
         [1.6441e+00, 2.7550e+00, 5.1603e-01, 2.9349e-01],
         [5.5659e+00, 2.2336e+00, 5.4313e-01, 3.1813e-01],
         ...,
         [1.0708e+00, 4.8314e-01, 2.3201e-01, 6.4090e-02],
         [1.1063e+00, 5.3486e-01, 2.2127e-01, 1.4596e-01],
         [3.8550e+00, 5.4772e-01, 2.1426e-01, 8.7654e-02]],

        [[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02],
         [7.6951e-01, 1.6432e+00, 5.3692e-01, 2.4189e-01],
         [1.0160e+00, 1.2573e+00, 5.3659e-01, 2.3823e-01],
         ...,
         [4.4885e+00, 5.9326e-01, 2.7911e-01, 1.7427e-01],
         [4.5558e+00, 5.3466e-01, 3.6408e-01, 9.3251e-02],
         [9.9958e-01, 5.5972e-01, 3.0972e-01, 3.4994e-01]]],
       grad_fn=<CatBa


Train Diffusion: 100%|██████████| 11/11 [00:31<00:00,  2.86s/it]


In [43]:
train(devi, l_r, niter, piter, batch_size, SDEFlow, ObsModel, csv_to_obs_df, 'AWB_synthetic_sol_df.csv', 0.1, state_dim_SAWB, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SAWB_SS, SAWB_SS_params_dict, temp_gen, temp_ref, analytical_steady_state_init_AWB)


Train Diffusion:   0%|          | 0/11 [00:00<?, ?it/s]

Moving average norm loss at 0 iterations is: 9000027385.8125. Best norm loss value is: 273858.125.

C_PATH mean = tensor([[0.5149, 0.6933, 0.8449, 1.0199],
        [0.5147, 0.6866, 0.8271, 1.0199]], grad_fn=<MeanBackward1>)

C_PATH = tensor([[[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02],
         [4.9025e-01, 3.4555e-01, 6.7835e-01, 6.2645e-01],
         [4.6546e-01, 4.0060e-01, 8.2979e-01, 1.3045e+00],
         ...,
         [4.7451e-01, 8.9870e-01, 7.7872e-01, 1.0778e+00],
         [3.4223e-01, 9.2241e-01, 1.1536e+00, 6.4982e-01],
         [3.1531e-01, 5.4660e-01, 5.5842e-01, 9.0073e-01]],

        [[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02],
         [4.9022e-01, 4.4540e-01, 8.1853e-01, 1.3949e+00],
         [4.6541e-01, 4.2551e-01, 8.6888e-01, 6.8453e-01],
         ...,
         [3.8802e-01, 7.4818e-01, 6.1177e-01, 7.7559e-01],
         [4.6812e-01, 4.3573e-01, 5.9754e-01, 1.1356e+00],
         [5.3969e-01, 6.5825e-01, 1.1098e+00, 4.7017e-01]]],
       grad_fn=<CatBackwar


Train Diffusion:  91%|█████████ | 10/11 [00:29<00:02,  2.83s/it]

Moving average ELBO loss at 10 iterations is: 5.0000000000001065e+19. Best ELBO loss value is: 1387660.625.

C_PATH mean = tensor([[1.7409, 0.8686, 0.5451, 0.3717],
        [1.7334, 0.8651, 0.5472, 0.3863]], grad_fn=<MeanBackward1>)

 C_PATH = tensor([[[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02],
         [6.6845e-01, 1.2110e+00, 5.9189e-01, 5.2553e-01],
         [2.0554e+00, 2.1238e+00, 8.1342e-01, 1.6718e-01],
         ...,
         [2.4523e+00, 5.1538e-01, 4.7106e-01, 6.2153e-01],
         [2.9393e+00, 4.9260e-01, 2.9262e-01, 3.6511e-01],
         [1.9846e+00, 6.5260e-01, 5.7467e-01, 5.0953e-01]],

        [[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02],
         [1.9296e+00, 2.3611e+00, 4.5677e-01, 2.4622e-01],
         [1.3215e+00, 1.2140e+00, 5.3784e-01, 5.4918e-01],
         ...,
         [8.0524e-01, 8.2999e-01, 2.9093e-01, 1.7694e-01],
         [1.0449e+00, 1.0270e+00, 5.4597e-01, 1.3540e-01],
         [8.7470e-01, 3.8501e-01, 2.5583e-01, 1.3141e-01]]],
       grad_fn=<


Train Diffusion: 100%|██████████| 11/11 [00:31<00:00,  2.90s/it]


In [44]:
train(devi, l_r, niter, piter, batch_size, SDEFlow, ObsModel, csv_to_obs_df, 'AWB_ECA_synthetic_sol_df.csv', 0.1, state_dim_SAWB, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SAWB_ECA_C, SAWB_ECA_C_params_dict, temp_gen, temp_ref, analytical_steady_state_init_AWB_ECA)


Train Diffusion:   0%|          | 0/11 [00:00<?, ?it/s]

Moving average norm loss at 0 iterations is: 9000027261.525. Best norm loss value is: 272615.25.

C_PATH mean = tensor([[0.6067, 0.6851, 0.8278, 1.0367],
        [0.6090, 0.6828, 0.8356, 1.0343]], grad_fn=<MeanBackward1>)

C_PATH = tensor([[[5.3606e+01, 1.9081e-01, 6.7073e-01, 1.3415e-02],
         [6.3485e-01, 6.1400e-01, 7.6927e-01, 7.3797e-01],
         [4.6950e-01, 8.6409e-01, 8.4031e-01, 8.1304e-01],
         ...,
         [7.5464e-01, 6.7258e-01, 1.0610e+00, 1.1461e+00],
         [6.8774e-01, 7.2470e-01, 7.5468e-01, 5.2559e-01],
         [5.3030e-01, 8.1238e-01, 1.0691e+00, 2.8655e-01]],

        [[5.3606e+01, 1.9081e-01, 6.7073e-01, 1.3415e-02],
         [6.3485e-01, 6.5485e-01, 7.5804e-01, 1.1755e+00],
         [4.0011e-01, 5.6324e-01, 6.5896e-01, 1.0278e+00],
         ...,
         [7.9753e-01, 9.2651e-01, 7.4436e-01, 1.0692e+00],
         [7.9639e-01, 7.8449e-01, 1.0987e+00, 3.1654e+00],
         [6.4381e-01, 6.7720e-01, 8.0059e-01, 3.3455e-01]]],
       grad_fn=<CatBackward>


Train Diffusion:  91%|█████████ | 10/11 [00:26<00:02,  2.72s/it]

Moving average ELBO loss at 10 iterations is: 5.000000000000068e+19. Best ELBO loss value is: 933075.3125.

C_PATH mean = tensor([[3.3854, 0.4935, 0.4342, 0.3417],
        [3.3464, 0.4978, 0.4326, 0.3418]], grad_fn=<MeanBackward1>)

 C_PATH = tensor([[[5.3606e+01, 1.9081e-01, 6.7073e-01, 1.3415e-02],
         [8.7361e-01, 2.8160e+00, 3.9193e-01, 2.4312e-01],
         [1.6057e+00, 2.0865e+00, 3.4390e-01, 3.8905e-01],
         ...,
         [3.5078e+00, 3.5395e-01, 2.9142e-01, 1.3126e-01],
         [3.8305e+00, 4.0646e-01, 2.3130e-01, 3.8743e-02],
         [3.2359e+00, 3.8016e-01, 5.9297e-01, 6.9783e-02]],

        [[5.3606e+01, 1.9081e-01, 6.7073e-01, 1.3415e-02],
         [1.3918e+00, 1.7887e+00, 5.5293e-01, 4.9244e-01],
         [2.5524e+00, 8.3801e-01, 4.5244e-01, 2.7256e-01],
         ...,
         [3.2627e+00, 4.7228e-01, 4.7456e-01, 3.3923e-01],
         [2.7533e+00, 3.4412e-01, 4.4653e-01, 1.0789e+00],
         [2.3513e+00, 3.8503e-01, 2.5897e-01, 1.0696e-01]]],
       grad_fn=<C


Train Diffusion: 100%|██████████| 11/11 [00:29<00:00,  2.66s/it]


In [45]:
train(devi, l_r, niter, piter, batch_size, SDEFlow, ObsModel, csv_to_obs_df, 'AWB_ECA_synthetic_sol_df.csv', 0.1, state_dim_SAWB, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SAWB_ECA_SS, SAWB_ECA_SS_params_dict, temp_gen, temp_ref, analytical_steady_state_init_AWB_ECA)


Train Diffusion:   0%|          | 0/11 [00:00<?, ?it/s]

Moving average norm loss at 0 iterations is: 9000026976.921875. Best norm loss value is: 269769.21875.

C_PATH mean = tensor([[0.6901, 0.5199, 0.6992, 0.7442],
        [0.6955, 0.5086, 0.6925, 0.7375]], grad_fn=<MeanBackward1>)

C_PATH = tensor([[[5.3606e+01, 1.9081e-01, 6.7073e-01, 1.3415e-02],
         [3.9304e-01, 4.2232e-01, 1.0812e+00, 5.8101e-01],
         [5.2352e-01, 2.8368e-01, 5.6906e-01, 7.1036e-01],
         ...,
         [6.8856e-01, 4.1288e-01, 7.0411e-01, 1.0186e+00],
         [8.4752e-01, 3.0050e-01, 7.1831e-01, 4.5525e-01],
         [5.0625e-01, 1.0979e+00, 7.6456e-01, 1.0971e+00]],

        [[5.3606e+01, 1.9081e-01, 6.7073e-01, 1.3415e-02],
         [3.9304e-01, 4.8733e-01, 6.1157e-01, 8.6844e-01],
         [2.1455e-02, 4.8617e-01, 1.2176e+00, 7.3027e-01],
         ...,
         [1.1496e+00, 6.7312e-01, 7.4989e-01, 1.2282e+00],
         [1.0482e+00, 5.5477e-01, 8.8441e-01, 7.1578e-01],
         [1.0771e+00, 5.5266e-01, 6.9577e-01, 8.2182e-01]]],
       grad_fn=<CatBac


Train Diffusion:  91%|█████████ | 10/11 [00:29<00:03,  3.05s/it]

Moving average ELBO loss at 10 iterations is: 5.000000000000634e+19. Best ELBO loss value is: 8930273.0.

C_PATH mean = tensor([[3.4345, 0.7792, 0.4018, 0.2008],
        [3.5112, 0.7822, 0.4012, 0.2029]], grad_fn=<MeanBackward1>)

 C_PATH = tensor([[[5.3606e+01, 1.9081e-01, 6.7073e-01, 1.3415e-02],
         [6.7668e-01, 5.2201e+00, 5.5515e-01, 3.4424e-01],
         [3.8250e+00, 1.6884e-01, 5.8764e-01, 3.5309e-01],
         ...,
         [6.5811e+00, 5.2455e-01, 3.4645e-01, 3.4987e-02],
         [3.8944e-01, 5.6786e-01, 2.0274e-01, 1.8349e-01],
         [1.3041e-01, 5.4453e-01, 2.9354e-01, 1.4687e-01]],

        [[5.3606e+01, 1.9081e-01, 6.7073e-01, 1.3415e-02],
         [1.3498e+00, 5.5324e-01, 6.0360e-01, 1.8385e-01],
         [2.1401e-01, 3.5347e+00, 5.3369e-01, 2.0978e-01],
         ...,
         [3.3374e-01, 6.0782e-01, 2.3981e-01, 3.8438e-01],
         [7.3347e+00, 5.7815e-01, 4.2845e-01, 7.6411e-02],
         [6.6302e+00, 5.7531e-01, 2.2375e-01, 9.1111e-02]]],
       grad_fn=<Cat


Train Diffusion: 100%|██████████| 11/11 [00:31<00:00,  2.90s/it]
